## Imports and Base models

In [13]:
from typing import Literal
from langchain_community.document_loaders import UnstructuredODTLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_core.pydantic_v1 import BaseModel, Field
from dotenv import load_dotenv
import os
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

True

In [2]:
gemini_token = os.getenv("GEMINI_TOKEN")
structurated_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=gemini_token)

In [3]:
hf_token = os.getenv("HF_TOKEN")
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=hf_token
)
chat_model = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\piamp\.cache\huggingface\token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Query Analysis

In [4]:
# Data Model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource"""

    datasource: Literal["vectorstore", "duckduckgo", "github", 'notify', 'none'] = Field(
        ...,
        description="Given a user question choose to route to the most relevant datasource"
    )

LLM_router = structurated_model.with_structured_output(RouteQuery)

In [5]:
# Prompt
system = """You are PampuAI, an expert at routing a user question to different tools.
The 'vectorstore' contains documents, including martin's cv and all actions that you can perform (PampuAI).
Use 'duckduckgo' for questions that can be answered by a web search.
Use 'github' for questions about martin's recent projects.
Use 'notify' to send a notification with a message to martin.
Use 'none' for chatting with PampuAI.
"""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | LLM_router

In [61]:
print(question_router.invoke({"question": "Que tecnologias maneja martin?"}))
print(question_router.invoke({"question": "Cual es el nombre de Obama?"}))
print(question_router.invoke({"question": "En que proyectos trabajo martin ultimamente?"}))
print(question_router.invoke({"question": "Decile a martin que me gusto su portfolio"}))
print(question_router.invoke({"question": "Que podes hacer?"}))
print(question_router.invoke({"question": "Hola!"}))

datasource='vectorstore'
datasource='duckduckgo'
datasource='github'
datasource='notify'
datasource='vectorstore'
datasource='none'


# Tools

## VectorStore (ADAPTATIVE RAG ¡IMPORTANT!)

### Retriever

In [6]:
file_path = (
    "content/MartinCV.odt"
)
loader = UnstructuredODTLoader(file_path)
pages = loader.load()

In [7]:
spliter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

chunks = spliter.split_documents(pages)

In [8]:
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma.from_documents(
    chunks,
    embedding=hf,
)
retriever = vectorstore.as_retriever()

### Revtrieval Grader

In [9]:
# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

structured_llm_grader = structurated_model.with_structured_output(GradeDocuments)

In [10]:
# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [12]:
question = "que tecnologias usa?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


### Generate

In [21]:
prompt = hub.pull("rlm/rag-prompt")

rag_chain = prompt | chat_model | StrOutputParser()

In [22]:
generation = rag_chain.invoke({"question": question, "context": docs})
print(generation)

Martin uses a range of technologies, including Python, C#, JavaScript, Django,.NET Framework, HTML5, CSS, Docker, Flask, SQL, and API REST.


### Hallucination Grader

In [24]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

LLM_grader = structurated_model.with_structured_output(GradeHallucinations)

In [25]:
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | LLM_grader

In [26]:
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='yes')

### Answer Grader

In [27]:
# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

LLM_grader = structurated_model.with_structured_output(GradeAnswer)

In [28]:
# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | LLM_grader

In [29]:
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='yes')

### Question Re-writer

In [39]:
system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning.
     Response only with the reformulated question."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)
question_rewriter = re_write_prompt | chat_model

In [42]:
question_rewriter.invoke({"question": "que tecnologias sabe usar martin?"})

AIMessage(content='What technologies is Martin proficient in?', response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=8, prompt_tokens=87, total_tokens=95), 'model': '', 'finish_reason': 'eos_token'}, id='run-d12bf378-6afd-4238-9cc8-a30c041b873d-0')

### Implementation (NOT FINAL, READING ABOUT LANGGRAPH NOW!!)

In [54]:
def route_vectorstore(question):

    while True:

        iterations = 0
        # Retrieve documents from vectorstore
        while True:
            if iterations >= 4:
                print("No relevant context found. Exiting...")
                generation = "Sorry, I couldn't find any relevant information to answer your question."
                return generation

            docs = retriever.invoke(question)
            relevant_documents = retrieval_grader.invoke({"question": question, "document": docs})
            if relevant_documents.binary_score == "yes":
                print('Relevant documents found.')
                break
            else:
                iterations += 1
                print("No relevant documents found. Rephrasing...")
                question = question_rewriter.invoke({"question": question}).content

        # Generate answer
        while True:
            generation = rag_chain.invoke({"question": question, "context": docs})
            hallucinations = hallucination_grader.invoke({"documents": docs, "generation": generation})
            if hallucinations.binary_score == "yes":
                print('Answer is grounded in the facts.')
                break
            else:
                print("Answer is not grounded in the facts. Generating new answer...")

        # Grade answer
        valid_answer = answer_grader.invoke({"question": question, "generation": generation})
        if valid_answer.binary_score == "yes":
            print('Answer addresses the question.')
            break
        else:
            question = question_rewriter.invoke({"question": question}).content
            print("Answer does not address the question. Generating new answer...")

    return generation

## DuckDuckGo

In [44]:
system = """
You are an assistant for question-answering tasks. Use only the provided context to answer the question.
Give the response in the same language as the question was asked.
"""

search_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Context: {context}\nQuestion: {question}"),
    ]
)

search = DuckDuckGoSearchResults()
def route_duckduckgo(question):
    results = search.invoke(question)
    chain = search_prompt | chat_model
    response = chain.invoke({"question": question, "context": results})
    return response.content

# Implementation 

Looking for a better implementation using langgraph and **langserve** !!!

In [57]:
while True:
    question = input("You: ")
    if question == "exit":
        break

    # Route the question
    router = question_router.invoke({"question": question})
    print("Selected tool:", router.datasource)

    if router.datasource == "notify":
        response = print("Sending notification...")

    elif router.datasource == "none":
        response = print("Chatting with PampuAI...")

    elif router.datasource == "vectorstore":
        response = route_vectorstore(question)

    elif router.datasource == "duckduckgo":
        response = route_duckduckgo(question)

    elif router.datasource == "github":
        response = print("Searching in github...")

    print("PampuAI:", response)

Selected tool: none
Chatting with PampuAI...
PampuAI: None
Selected tool: vectorstore
Relevant documents found.
Answer is grounded in the facts.
Answer addresses the question.
PampuAI: Martin maneja las tecnologías de backend Python, Django,.NET Framework, Flask, y FastAPI.
Selected tool: vectorstore
Relevant documents found.
Answer is grounded in the facts.
Answer addresses the question.
PampuAI: Martin has no formal work experience, but he works as a freelance developer from January 2024 to present. He has worked on several projects, including a contract management system and a stock control system, as part of a development team. He has also participated in a "No Country" simulation, developing a fully functional application from scratch.
Selected tool: vectorstore
Relevant documents found.
Answer is grounded in the facts.
Answer addresses the question.
PampuAI: Martin has experience developing a contract management system, which demonstrates his ability to create complex software an